https://www.kaggle.com/datasets/ninzaami/loan-predication

In [1]:
import kagglehub
import pandas as pd

/Users/lallu/Downloads/Repositories/Personal/DataScienceProjectForBeginners/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
path = kagglehub.dataset_download("ninzaami/loan-predication")

In [3]:
print("Path to dataset files:", path)

Path to dataset files: /Users/lallu/.cache/kagglehub/datasets/ninzaami/loan-predication/versions/1


In [4]:
df = pd.read_csv("loandata.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
# shape of the dataset
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 614, Columns: 13


In [6]:
# summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [7]:
# quick stats for numeric and object columns
print(df.describe(include='all'))

         Loan_ID Gender Married Dependents Education Self_Employed  \
count        614    601     611        599       614           582   
unique       614      2       2          4         2             2   
top     LP001002   Male     Yes          0  Graduate            No   
freq           1    489     398        345       480           500   
mean         NaN    NaN     NaN        NaN       NaN           NaN   
std          NaN    NaN     NaN        NaN       NaN           NaN   
min          NaN    NaN     NaN        NaN       NaN           NaN   
25%          NaN    NaN     NaN        NaN       NaN           NaN   
50%          NaN    NaN     NaN        NaN       NaN           NaN   
75%          NaN    NaN     NaN        NaN       NaN           NaN   
max          NaN    NaN     NaN        NaN       NaN           NaN   

        ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count        614.000000         614.000000  592.000000         600.00000   
unique 

In [8]:
# count of missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)

Credit_History      50
Self_Employed       32
LoanAmount          22
Dependents          15
Loan_Amount_Term    14
Gender              13
Married              3
dtype: int64


In [9]:
# fill categorical columns with mode
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df.fillna({col: df[col].mode()[0]}, inplace=True)

# fill numerical columns with median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df.fillna({col: df[col].median()}, inplace=True)

In [10]:
# example: convert 'LoanAmount' to float
df['LoanAmount'] = df['LoanAmount'].astype(float)

# convert categorical variables to category type
for col in cat_cols:
    df[col] = df[col].astype('category')

In [11]:
# strip whitespace and convert to consistent case
for col in cat_cols:
    df[col] = df[col].str.strip().str.lower()

In [12]:
# capping outliers at 99th percentile
for col in ['LoanAmount', 'ApplicantIncome']:
    upper = df[col].quantile(0.99)
    df[col] = df[col].apply(lambda x: upper if x > upper else x)

In [13]:
df.to_csv('cleaned_loan_data.csv', index=False)

In [14]:
def clean_loan_data(df):
    # fill missing values
    for col in df.select_dtypes(include='object'):
        df.fillna({col: df[col].mode()[0]}, inplace=True)
    for col in df.select_dtypes(include=['int64', 'float64']):
        df.fillna({col: df[col].median()}, inplace=True)

    # type conversions
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].astype('category')
        df[col] = df[col].str.strip().str.lower()

    # outlier capping
    for col in ['LoanAmount', 'ApplicantIncome']:
        if col in df.columns:
            upper = df[col].quantile(0.99)
            df[col] = df[col].apply(lambda x: upper if x > upper else x)

    return df

In [15]:
df = pd.read_csv('loandata.csv')
df_clean = clean_loan_data(df)

### Summary

So, even though not all datasets require machine learning, every dataset does demand proper cleaning, and this pipeline sets the foundation for accurate, reliable analysis and modelling. I hope you liked this article on building a data cleaning pipeline using Pandas. 